In [1]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak
import pickle

import torch
import torch.nn.functional as F

In [2]:
with uproot.open("../pythia/output/dataset_diHiggs_mu60_NumEvents10k_MinJetpT25.root:fastjet") as f:
    jet_pt = f["jet_pt"].array()
    jet_eta = f["jet_eta"].array()
    jet_phi = f["jet_phi"].array()
    jet_corrJVF = f["jet_corrJVF"].array()
    jet_RpT = f["jet_RpT"].array()
    jet_m = f["jet_m"].array()
    trk_pt = f["trk_pT"].array()
    trk_eta = f["trk_eta"].array()
    trk_phi = f["trk_phi"].array()
    trk_q = f["trk_q"].array()
    trk_d0 = f["trk_d0"].array()
    trk_z0 = f["trk_z0"].array()
    trk_label = f["trk_label"].array()
    jet_trk_IDX = f["jet_track_index"].array()
    jet_pufr_truth = f["jet_pufr_truth"].array()

In [3]:
%%time
num_events = len(jet_pt)
trk_feats = []
for event in range(num_events):
    if event%5==0:
        print("Processing: ", event, " / ", num_events, end="\r")
    idx_list = list(jet_trk_IDX[event])
    idx_list.append(len(trk_pt[event]))
    
    jet_trk_feats = []
    for i in range(len(idx_list)-1):
        start_idx = idx_list[i]
        end_idx = idx_list[i+1]
        trk_pt_tmp = np.array(trk_pt[event][start_idx:end_idx])
        trk_eta_tmp = np.array(trk_eta[event][start_idx:end_idx])
        trk_phi_tmp = np.array(trk_phi[event][start_idx:end_idx])
        trk_q_tmp = np.array(trk_q[event][start_idx:end_idx])
        trk_d0_tmp = np.array(trk_d0[event][start_idx:end_idx])
        trk_z0_tmp = np.array(trk_z0[event][start_idx:end_idx])
        trk_label_tmp = np.array(trk_label[event][start_idx:end_idx])

        feats = [trk_pt_tmp, trk_eta_tmp, trk_phi_tmp, trk_q_tmp,
                trk_d0_tmp, trk_z0_tmp, trk_label_tmp]
        feats = np.stack(feats, axis=-1)
        jet_trk_feats.append(feats)
    
    trk_feats.append(jet_trk_feats)
    
trk_feats = ak.Array(trk_feats)

print("Processing: ", num_events, " / ", num_events)
print("Num Events: ", len(trk_feats))
print("Num Jets in first event: ", len(trk_feats[0]))
print("Num Tracks in first event first jet: ", len(trk_feats[0][0]))
print("Num Tracks features: ", len(trk_feats[0][0][0]))

Processing:  10000  /  10000
Num Events:  10000
Num Jets in first event:  109
Num Tracks in first event first jet:  140
Num Tracks features:  7
CPU times: user 6min 24s, sys: 2.93 s, total: 6min 27s
Wall time: 6min 26s


In [4]:
%%time
num_events = len(jet_pt)
jet_feats = []
for event in range(num_events):
    jet_corrJVF_tmp = np.array(jet_corrJVF[event])
    jet_RpT_tmp = np.array(jet_RpT[event])
    jet_pt_tmp = np.array(jet_pt[event])
    jet_eta_tmp = np.array(jet_eta[event])
    jet_phi_tmp = np.array(jet_phi[event])
    jet_m_tmp = np.array(jet_m[event])
    jet_pufr_truth_tmp = np.array(jet_pufr_truth[event])

    feats = [jet_pt_tmp, jet_eta_tmp, jet_phi_tmp, jet_m_tmp, jet_corrJVF_tmp, jet_RpT_tmp, jet_pufr_truth_tmp]
    feats = np.stack(feats, axis=-1)
    
    jet_feats.append(feats)
    
jet_feats = ak.Array(jet_feats)

print("Num Events: ", len(jet_feats))
print("Num Jets in first event: ", len(jet_feats[0]))
print("Num Jet Features: ", len(jet_feats[0][0]))

Num Events:  10000
Num Jets in first event:  109
Num Jet Features:  7
CPU times: user 17.1 s, sys: 63.2 ms, total: 17.2 s
Wall time: 17.1 s


In [5]:
jet_mask = abs(jet_feats[:,:,1])<4
selected_jets = jet_feats[jet_mask]
selected_tracks = trk_feats[jet_mask]

trk_q_cut = selected_tracks[:,:,:,3]!=0          # Skip neutral particles
trk_eta_cut = abs(selected_tracks[:,:,:,1])<4    # Skip forward region
trk_pt_cut = selected_tracks[:,:,:,0]>0.4        # 400MeV Cut

mask = trk_q_cut & trk_eta_cut & trk_pt_cut

refined_tracks = selected_tracks[mask]

all_tracks = ak.flatten(refined_tracks, axis=2)

print("Jet Shape:\t", selected_jets.type)
print("Trk_Jet  Shape:\t", refined_tracks.type)
print("Trk_All Shape:\t", all_tracks.type)

Jet Shape:	 10000 * var * var * float64
Trk_Jet  Shape:	 10000 * var * var * var * float64
Trk_All Shape:	 10000 * var * var * float64


In [6]:
# Generate List of torch tensors on event by event basis
# Pad number of tracks per jet 

num_events = len(selected_jets)

Event_Data = []
Event_Labels = []

for event in range(num_events):
    if event%5==0:
        print("Processing: ", event, " / ", num_events, end="\r")
    jets = torch.Tensor(selected_jets[event,:,:])
    
    num_trks = ak.num(refined_tracks[event], axis=1)
    max_num_trks = ak.max(num_trks)

    trk_list = []
    num_jets = len(selected_jets[event])
    for jet in range(num_jets):
        tracks = torch.Tensor(refined_tracks[event][jet,:])
        pad = (0,0,0,max_num_trks-len(tracks))
        tracks = F.pad(tracks,pad)
        trk_list.append(torch.unsqueeze(tracks,dim=0))
    tracks = torch.cat(trk_list,dim=0)
    
    # Append all data but don't include label 0:-1!
    flat_tracks = torch.Tensor(all_tracks[event][:,0:-1])
    Event_Data.append((jets[:,0:-1],tracks[:,:,0:-1],flat_tracks))
    #Event_Labels.append((jets[:,-1].reshape(-1,1),tracks[:,:,-1].reshape(-1,1)))
    Event_Labels.append(jets[:,-1].reshape(-1,1))

print("Processing: ", num_events, " / ", num_events)

Processing:  10000  /  10000


In [7]:
train_split = int(0.7*num_events)  # 70% train
test_split = int(0.75*num_events)  #  5% val + 25% test

Event_List = list(zip(Event_Data, Event_Labels))

Events_training = Event_List[0:train_split]
Events_validation = Event_List[train_split:test_split]
Events_testing = Event_List[test_split:]

print("Processing: ", num_events, " / ", num_events)
print("Training Events: ", len(Events_training))
print("Validation Events: ", len(Events_validation))
print("Testing Events: ", len(Events_testing))

Processing:  10000  /  10000
Training Events:  7000
Validation Events:  500
Testing Events:  2500


In [8]:
X_train, y_train = list(zip(*Events_training))
X_val, y_val = list(zip(*Events_validation))
X_test, y_test = list(zip(*Events_testing))

data = (X_train, y_train, X_val, y_val, X_test, y_test)

pickle.dump(data, open("data.pkl", "wb"))

In [9]:
print("X_train Indices Reference:")
print("\tNum Events: ", len(X_train))
print("\tNum Tensors: ", len(X_train[0]), "(Jet, trk-jet, flat trk)")
print("\tNum Jets: ", len(X_train[0][0]))
print("\tNum Trks per Jet: ", len(X_train[0][1][0]))
print("\tNum Flat Trks: ", len(X_train[0][2]))
print("\tNum Jet Feats: ", len(X_train[0][0][0]))
print("\tNum Trk Feats: ", len(X_train[0][1][0][0]))
print()
print("y_train Indices Reference:")
print("\tNum Events: ", len(y_train))
print("\tNum Jets: ", len(y_train[0]))

X_train Indices Reference:
	Num Events:  7000
	Num Tensors:  3 (Jet, trk-jet, flat trk)
	Num Jets:  80
	Num Trks per Jet:  48
	Num Flat Trks:  1788
	Num Jet Feats:  6
	Num Trk Feats:  6

y_train Indices Reference:
	Num Events:  7000
	Num Jets:  80


In [10]:
# Shuffle
# Split train, val, test
# Calc train mean and std
# Norm train, val, test
# Split batches
# Pad jets and tracks in batches

In [11]:
"""
# Shuffle and pad the tracks and jets
num_events = len(selected_jets)

for event in range(1):
    num_jets = len(selected_jets[event])
    num_trks = ak.num(refined_tracks[event], axis=1)
    max_num_trks = ak.max(num_trks)
    print(max_num_trks)
    
    
    for jet in range(num_jets):
        p = np.random.permutation(len(refined_tracks[event][jet]))
        tmp = refined_tracks[event][jet][p]
        #print(tmp.show())
        tmp = ak.pad_none(tmp,max_num_trks,axis=0)
        tmp = ak.pad_none(tmp,7,axis=1)
        tmp = ak.fill_none(tmp, 0)
        print(jet)
        print(tmp.show())

    p = np.random.permutation(len(selected_jets[event]))
    tmp = selected_jets[event][p]
    tmp = refined_tracks[event][p]
    
# Pad num tracks per jet on event basis
num_events = len(selected_jets)
print(num_events)
for event in range(1):
    len_list = []
    num_jets = len(selected_jets[event])
    num_trks = ak.num(refined_tracks[event], axis=1)
    max_num_trks = ak.max(num_trks)
    print(max_num_trks)
    
    print(refined_tracks[event].show)
    pad = ak.pad_none(refined_tracks[event], max_num_trks, axis=1).to_list()
    print(pad[1][-1])
    pad2 = ak.fill_none(pad,0).to_list()
    #print(pad2)
    print(pad2[1][-1])

    pad = ak.pad_none(pad, 7, axis=1)
    print(pad)
    print(pad[-1])
    tmp = ak.pad_none(pad,7,axis=1)
    print(pad)
    print(pad[-1])
"""

'\n# Shuffle and pad the tracks and jets\nnum_events = len(selected_jets)\n\nfor event in range(1):\n    num_jets = len(selected_jets[event])\n    num_trks = ak.num(refined_tracks[event], axis=1)\n    max_num_trks = ak.max(num_trks)\n    print(max_num_trks)\n    \n    \n    for jet in range(num_jets):\n        p = np.random.permutation(len(refined_tracks[event][jet]))\n        tmp = refined_tracks[event][jet][p]\n        #print(tmp.show())\n        tmp = ak.pad_none(tmp,max_num_trks,axis=0)\n        tmp = ak.pad_none(tmp,7,axis=1)\n        tmp = ak.fill_none(tmp, 0)\n        print(jet)\n        print(tmp.show())\n\n    p = np.random.permutation(len(selected_jets[event]))\n    tmp = selected_jets[event][p]\n    tmp = refined_tracks[event][p]\n    \n# Pad num tracks per jet on event basis\nnum_events = len(selected_jets)\nprint(num_events)\nfor event in range(1):\n    len_list = []\n    num_jets = len(selected_jets[event])\n    num_trks = ak.num(refined_tracks[event], axis=1)\n    max_nu